# Project 4: Predict West Nile Virus

**Group 1: ['Harry', 'Celestine', 'YX', 'Xiu Ting']**

Predict probability (not just ones and zeroes)

[Kaggle link](https://www.kaggle.com/c/predict-west-nile-virus/)


stop collab from disconnecting: 
function KeepClicking(){
console.log("Clicking");
document.querySelector("colab-connect-button").click()
}
setInterval(KeepClicking,60000)

# Background
# Problem Statement
# Contents
# Data Dictionary 

# Import Libraries

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns 
import datetime
!pip install suntime
from suntime import Sun, SunTimeException
from pytz import timezone
import pytz


# Import Datasets

In [ ]:
# Import datasets
train_df = pd.read_csv('train.csv')
weather_df = pd.read_csv('weather.csv')

# Data Cleaning

## Weather
### Summary of Changes
- Imputed missing values for daily average temperature Tavg with average of Tmax and Tmin
- Calculated missing values for Cool and Warm with Tavg
- Calculated missing Depart from Station 2 with 30 year normal temperature based on Station 1 readings and Station 2 Tavg.
- Imputed missing values for WetBulb, PrecipTotal, StnPressure, SeaLevel, AvgSpeed using readings of Station with non-missing value.
- Imputed 'T' or trace values as 0.01.
- Imputed Sunrise and Sunset for Station 2 with Station 1 values
- Split and recombined CodeSum to create proper spacing between different codes
- Created new feature counting number of exceptional weather phenomena based on CodeSum
- Created more interpretable features like rain and lowvis based on CodeSum
- Changed Date from string object to datetime64
- Dropped Water1, Depth, SnowFall due to high missing values (>99.5% 'M' or 0)
- Transformed all features into float values
- Merged Station 1 and Station 2 by averaging values of each station
- Added Year, Month, Week and Day of Week features


In [ ]:
weather_df.columns

Index(['Station', 'Date', 'Tmax', 'Tmin', 'Tavg', 'Depart', 'DewPoint',
       'WetBulb', 'Heat', 'Cool', 'Sunrise', 'Sunset', 'CodeSum', 'Depth',
       'Water1', 'SnowFall', 'PrecipTotal', 'StnPressure', 'SeaLevel',
       'ResultSpeed', 'ResultDir', 'AvgSpeed'],
      dtype='object')

In [ ]:
weather_df.head()

,Station,Date,Tmax,Tmin,Tavg,Depart,DewPoint,WetBulb,Heat,Cool,Sunrise,Sunset,CodeSum,Depth,Water1,SnowFall,PrecipTotal,StnPressure,SeaLevel,ResultSpeed,ResultDir,AvgSpeed
0,1,2007-05-01,83,50,67,14,51,56,0,2,0448,1849,,0,M,0.0,0.00,29.10,29.82,1.7,27,9.2
1,2,2007-05-01,84,52,68,M,51,57,0,3,-,-,,M,M,M,0.00,29.18,29.82,2.7,25,9.6
2,1,2007-05-02,59,42,51,-3,42,47,14,0,0447,1850,BR,0,M,0.0,0.00,29.38,30.09,13.0,4,13.4
3,2,2007-05-02,60,43,52,M,42,47,13,0,-,-,BR HZ,M,M,M,0.00,29.44,30.08,13.3,2,13.4
4,1,2007-05-03,66,46,56,2,40,48,9,0,0446,1851,,0,M,0.0,0.00,29.39,30.12,11.7,7,11.9


In [ ]:
weather_df.isnull().sum()

Station        0
Date           0
Tmax           0
Tmin           0
Tavg           0
Depart         0
DewPoint       0
WetBulb        0
Heat           0
Cool           0
Sunrise        0
Sunset         0
CodeSum        0
Depth          0
Water1         0
SnowFall       0
PrecipTotal    0
StnPressure    0
SeaLevel       0
ResultSpeed    0
ResultDir      0
AvgSpeed       0
dtype: int64

In [ ]:
weather_df['PrecipTotal'].value_counts()

0.00    1577
  T      318
0.01     127
0.02      63
0.03      46
        ... 
1.26       1
3.66       1
1.19       1
1.60       1
1.48       1
Name: PrecipTotal, Length: 168, dtype: int64

In [ ]:
weather_df.replace('-', np.nan, inplace = True)
weather_df.replace('M', np.nan, inplace = True)

In [ ]:
weather_df.head()

,Station,Date,Tmax,Tmin,Tavg,Depart,DewPoint,WetBulb,Heat,Cool,Sunrise,Sunset,CodeSum,Depth,Water1,SnowFall,PrecipTotal,StnPressure,SeaLevel,ResultSpeed,ResultDir,AvgSpeed
0,1,2007-05-01,83,50,67,14,51,56,0,2,0448,1849,,0,NaN,0.0,0.00,29.10,29.82,1.7,27,9.2
1,2,2007-05-01,84,52,68,NaN,51,57,0,3,NaN,NaN,,NaN,NaN,NaN,0.00,29.18,29.82,2.7,25,9.6
2,1,2007-05-02,59,42,51,-3,42,47,14,0,0447,1850,BR,0,NaN,0.0,0.00,29.38,30.09,13.0,4,13.4
3,2,2007-05-02,60,43,52,NaN,42,47,13,0,NaN,NaN,BR HZ,NaN,NaN,NaN,0.00,29.44,30.08,13.3,2,13.4
4,1,2007-05-03,66,46,56,2,40,48,9,0,0446,1851,,0,NaN,0.0,0.00,29.39,30.12,11.7,7,11.9


In [ ]:
weather_df.isna().sum() * 100 / len(weather_df)

Station          0.000000
Date             0.000000
Tmax             0.000000
Tmin             0.000000
Tavg             0.373641
Depart          50.000000
DewPoint         0.000000
WetBulb          0.135870
Heat             0.373641
Cool             0.373641
Sunrise         50.000000
Sunset          50.000000
CodeSum          0.000000
Depth           50.000000
Water1         100.000000
SnowFall        50.000000
PrecipTotal      0.067935
StnPressure      0.135870
SeaLevel         0.305707
ResultSpeed      0.000000
ResultDir        0.000000
AvgSpeed         0.101902
dtype: float64

### PrecipTotal
'T' which represents 'Trace', denotes that there is an amount of precipitation, which is greater than zero but is too small to be measured. The lowest value in PrecipTotal is 0 and the second lowest is 0.01, hence we will impute 'T' with the mean of these 2 numbers, which is 0.005.

In [ ]:
weather_df['PrecipTotal'].value_counts(ascending = False)

0.00    1577
  T      318
0.01     127
0.02      63
0.03      46
        ... 
2.17       1
1.75       1
3.07       1
1.04       1
0.62       1
Name: PrecipTotal, Length: 167, dtype: int64

In [ ]:
weather_df.loc[weather_df['PrecipTotal'] == '  T', 'PrecipTotal'] = 0.005

In [ ]:
weather_df['PrecipTotal'].isnull().sum()

2

In [ ]:
weather_df[weather_df['PrecipTotal'].isna()]

,Station,Date,Tmax,Tmin,Tavg,Depart,DewPoint,WetBulb,Heat,Cool,Sunrise,Sunset,CodeSum,Depth,Water1,SnowFall,PrecipTotal,StnPressure,SeaLevel,ResultSpeed,ResultDir,AvgSpeed
117,2,2007-06-28,73,61,67,NaN,56,61,0,2,NaN,NaN,,NaN,NaN,NaN,NaN,29.43,30.07,12.2,2,13.3
119,2,2007-06-29,71,56,64,NaN,56,60,1,0,NaN,NaN,,NaN,NaN,NaN,NaN,29.47,30.11,7.4,2,8.2


In [ ]:
weather_df['PrecipTotal'].value_counts(normalize = True)

0.00     0.536030
0.005    0.108090
0.01     0.043168
0.02     0.021414
0.03     0.015636
           ...   
2.17     0.000340
1.75     0.000340
3.07     0.000340
1.04     0.000340
0.62     0.000340
Name: PrecipTotal, Length: 167, dtype: float64

There are still observations with NaN Values in PrecipTotal. As more than 53% of the observations have a PrecipTotal of 0, we will impute these 2 NaN values with 0.

In [ ]:
weather_df['PrecipTotal'] = weather_df['PrecipTotal'].fillna(0)

In [ ]:
weather_df['PrecipTotal'].isnull().sum()

0

# Sunrise and Sunset
Sunrise's and sunset's values of station 2 are missing. We will be calculating them using the suntime module and adjusting them with the daylight saving calculation. Codes adapted from: [source](https://stackoverflow.com/questions/38986527/sunrise-and-sunset-time-in-python/38986561)

In [ ]:
# Convert date to datetime
weather_df['Date'] = pd.to_datetime(weather_df['Date'])

In [ ]:
weather_df['Date'][0]

Timestamp('2007-05-01 00:00:00')

In [ ]:
latitude = 41.786
longitude = -87.752

sun = Sun(latitude, longitude)

central = timezone('US/Central')

sunrise_list = []
sunset_list = []
for value in weather_df['Date']:
    abd = value
    abd_sr = sun.get_local_sunrise_time(value, local_time_zone=central)
    abd_sr = abd_sr - abd_sr.dst()
    abd_ss = sun.get_local_sunset_time(value, local_time_zone=central)
    abd_ss = abd_ss - abd_ss.dst()
    sunrise_list.append(abd_sr.strftime('%H%M'))
    sunset_list.append(abd_ss.strftime('%H%M'))

In [ ]:
index = weather_df.loc[weather_df['Station'] == 2].index

for i in index:
    weather_df.loc[i, 'Sunrise'] = sunrise_list[i]
    weather_df.loc[i, 'Sunset'] = sunset_list[i]

In [ ]:
# Check updated fram
weather_df[['Sunrise', 'Sunset']][weather_df['Station'] == 2]

,Sunrise,Sunset
1,0448,1849
3,0447,1850
5,0445,1851
7,0444,1852
9,0443,1853
...,...,...
2935,0617,1652
2937,0618,1651
2939,0620,1649
2941,0621,1648


In [ ]:
!wget

In [ ]:
from google.colab import files
uploaded = files.upload()

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# interactive dataframes? 

%load_ext google.colab.data_table

# EDA & Feature Creation

# New Section

# Modelling